# Basic Sellar Example

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DKilkenny/OpenMDAO/blob/notebook_testing/openmdao/docs/notebooks/Basic_Sellar_Example.ipynb)

In [1]:
import openmdao.api as om
import numpy as np
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives
from openmdao.utils.assert_utils import assert_near_equal

In [2]:
prob = om.Problem()
model = prob.model

model.add_subsystem('px', om.IndepVarComp('x', 1.0), promotes=['x'])
model.add_subsystem('pz', om.IndepVarComp('z', np.array([5.0, 2.0])), promotes=['z'])

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                        z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

prob.model.nonlinear_solver = om.NonlinearBlockGS()

model.approx_totals(method='fd', step=1e-5)

prob.setup()
prob.set_solver_print(level=0)
prob.run_model()

In [3]:
assert_near_equal(prob['y1'], 25.58830273, .00001)
assert_near_equal(prob['y2'], 12.05848819, .00001)

3.266469240658292e-09

In [4]:
wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')

In [5]:
assert_near_equal(J['obj', 'z'][0][0], 9.61001056, .00001)
assert_near_equal(J['obj', 'z'][0][1], 1.78448534, .00001)

1.6028959844473032e-07